<a href="https://colab.research.google.com/github/w-lly/Agentic-AI-for-Insights/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello World")